In [27]:
from keras.models import Model
from keras.layers import Input, Dense
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.optimizers import Adam



In [23]:
"""
-- Load data from analysis dataset --
"""

parent_folder_path = "images/analysis_dataset/"

categories = {}

# Iterate over the contents of the "original_dataset" folder
for index, folder_name in enumerate(sorted(os.listdir(parent_folder_path))):
    folder_path = os.path.join(parent_folder_path, folder_name)
    if os.path.isdir(folder_path):
        categories[folder_name] = index

category_amount = []

# get the category with the least images
for category in categories.keys():
    folder_path = os.path.join(parent_folder_path, category)
    image_files = os.listdir(folder_path)
    category_amount.append(len(image_files))

max_training = min(category_amount)

arrays = []
# convert the images into a pytorch dataset
for cat_folder, value in categories.items():

    folder_path = os.path.join(parent_folder_path, cat_folder)
    image_files = os.listdir(folder_path)

    for i, file_name in enumerate(image_files):

        if i >= max_training: break
        file_path = os.path.join(folder_path, file_name)
        image = Image.open(file_path)
        image_array = np.array(image)

        # verify all images are of the desired size
        if image.size != (100, 100):
            print(file_path, " IS NOT 250x250, it is: ", image.size)
            continue

        if image_array.shape != (250, 250):
            image_array = np.dot(image_array[..., :3], [0.2989, 0.5870, 0.1140])

        arrays.append(image_array)

# Normalize the array of the normal 
arrays = arrays/ np.max(arrays)

arrays_labels = []
for i in range(len(categories)):
    arrays_labels += [i] * max_training

arrays_labels_normal = np.array(arrays_labels)


In [52]:
"""
-- Load autoencoder, train and testing data --

Using the distribution:
80% -> autoencoder
10% -> training data
10% -> testing data
"""

X_auto, X, y_auto, y = train_test_split(arrays, arrays_labels, test_size=0.5, random_state=42, stratify=arrays_labels)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)

y_train = to_categorical(y_train, num_classes=38)
y_test = to_categorical(y_test, num_classes=38)

X_auto = np.array(X_auto)
X_auto = X_auto.reshape((len(X_auto), -1))

X_train = np.array(X_train)
X_train = X_train.reshape((len(X_train), -1))

X_test = np.array(X_test)
X_test = X_test.reshape((len(X_test), -1))

X = np.array(X)
X = X.reshape((len(X), -1))

In [57]:


# definir la arquitectura del autoencoder

input_img = Input(shape=(10000,))
encoded = Dense(500, activation='relu')(input_img)
encoded = Dense(400, activation='relu')(encoded)
encoded = Dense(300, activation='relu')(encoded)
latent = Dense(200, activation='relu')(encoded)
encoded = Dense(300, activation='relu')(encoded)
decoded = Dense(400, activation='relu')(latent)
decoded = Dense(500, activation='relu')(decoded)
decoded = Dense(10000, activation='sigmoid')(decoded)

# Crear el modelo del autoencoder
autoencoder = Model(input_img, decoded)

# Compilar el modelo
optimizer = Adam(learning_rate = 0.01)
autoencoder.compile(optimizer=optimizer, loss='binary_crossentropy')

print("Before fit autoencoder")

# Entrenar el autoencoder
autoencoder.fit(X_auto, X_auto,
                epochs=30,
                batch_size=256,
                shuffle=True,
                validation_data=(X, X))

print("after autoencoder")
# Obtener el vector latente (representación codificada)
encoder = Model(input_img, latent)
encoded_train = encoder.predict(X_train)
encoded_test = encoder.predict(X_test)



Before fit autoencoder
Epoch 1/30
86/86 [==============================] - 27s 292ms/step - loss: 0.6777 - val_loss: 0.6685
Epoch 2/30
86/86 [==============================] - 18s 208ms/step - loss: 0.6687 - val_loss: 0.6683
Epoch 3/30
86/86 [==============================] - 18s 207ms/step - loss: 0.6685 - val_loss: 0.6683
Epoch 4/30
86/86 [==============================] - 18s 208ms/step - loss: 0.6685 - val_loss: 0.6683
Epoch 5/30
86/86 [==============================] - 18s 208ms/step - loss: 0.6685 - val_loss: 0.6683
Epoch 6/30
60/86 [===================>..........] - ETA: 4s - loss: 0.6686

KeyboardInterrupt: 

In [54]:
# Definir la arquitectura del clasificador utilizando el vector latente como entrada
input_latent = Input(shape=(200,))
output = Dense(128, activation='softmax')(input_latent)
output = Dense(64, activation='softmax')(input_latent)
output = Dense(38, activation='softmax')(input_latent)

# Crear el modelo del clasificador
classifier = Model(input_latent, output)

# Compilar el modelo del clasificador
optimizer = Adam(learning_rate = 0.0001)
classifier.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Entrenar el clasificador utilizando el vector latente como características de entrada
classifier.fit(encoded_train, y_train,
               epochs=200,
               batch_size=256,
               validation_data=(encoded_test, y_test), verbose=1)

# Evaluar el rendimiento del clasificador en el conjunto de prueba
loss, accuracy = classifier.evaluate(encoded_test, y_test)
print("Pérdida de clasificación:", loss)
print("Exactitud de clasificación:", accuracy)

Epoch 1/200
73/73 [==============================] - 2s 11ms/step - loss: 3.6411 - accuracy: 0.0263 - val_loss: 3.6407 - val_accuracy: 0.0262
Epoch 2/200
73/73 [==============================] - 0s 6ms/step - loss: 3.6406 - accuracy: 0.0263 - val_loss: 3.6403 - val_accuracy: 0.0262
Epoch 3/200
73/73 [==============================] - 0s 6ms/step - loss: 3.6403 - accuracy: 0.0263 - val_loss: 3.6400 - val_accuracy: 0.0262
Epoch 4/200
73/73 [==============================] - 0s 5ms/step - loss: 3.6400 - accuracy: 0.0263 - val_loss: 3.6397 - val_accuracy: 0.0262
Epoch 5/200
73/73 [==============================] - 0s 6ms/step - loss: 3.6397 - accuracy: 0.0263 - val_loss: 3.6395 - val_accuracy: 0.0262
Epoch 6/200
73/73 [==============================] - 0s 6ms/step - loss: 3.6394 - accuracy: 0.0263 - val_loss: 3.6392 - val_accuracy: 0.0262
Epoch 7/200
73/73 [==============================] - 0s 6ms/step - loss: 3.6392 - accuracy: 0.0263 - val_loss: 3.6390 - val_accuracy: 0.0262
Epoch 8/200


KeyboardInterrupt: 